<a href="https://colab.research.google.com/github/MarogobaCTD/Gemini/blob/main/Projeto_Imersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ipywidgets==8.1.2

In [ ]:
#Importação das bibliotecas e autenticação
import os
import io
import pandas as pd
import numpy as np
import threading
import time
from typing import Any
import ipywidgets as widgets
import google.generativeai as genai
from IPython.display import display
from google.colab import userdata
from google.colab import output
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)


In [ ]:
# Informações para o modelo
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 0,
  "max_output_tokens": 2048,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.0-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [ ]:
def pega_arquivo(nomeArquivo):
      if not os.path.lexists(nomeArquivo):
          return nomeArquivo
      counter = 1
      while True:
          path, ext = os.path.splitext(nomeArquivo)
          novo_nome = '{} ({}){}'.format(path, counter, ext)
          if not os.path.lexists(novo_nome):
              return novo_nome
          counter += 1

In [ ]:
def upload_arquivo():

  def on_button_clicked(change):
      try:
          uploaded_file.disabled = True
          #recuperando as informações do arquivo
          arquivo = change['new']
          #print(arquivo)
          if arquivo:
            if arquivo['_counter'] == 1:
              chave_nome = arquivo['metadata'][0]['name']
              #lista_arquivo.pop(chave_nome)
              total = arquivo['metadata'][0]['size']
              #verificando se já existe um arquivo com o mesmo nome
              #e gravar um seguencial
              nome_arquivo = pega_arquivo(chave_nome)
              if (nome_arquivo != chave_nome):
                  print('Saved {antigo} to {novo}'.format(
                      antigo=chave_nome, novo=os.getcwd() + '/' + nome_arquivo))
              #preparando a progressão para agravação do arquivo
              prog = widgets.IntProgress(value=0,
                                          min=0,
                                          max=total,
                                          description = 'Progresso: ',
                                          bar_style='success',
                                          orientation='horizontal')
              display(prog)
              dados = arquivo['data'][0]
              #Criando o arquivo localmente
              with open(nome_arquivo, 'wb') as f:
                for i in range(0, len(dados), 2048):
                  # Obtenha um pedaço do conteúdo
                  data = dados[i:i+2048]
                  f.write(data)
                  f.flush()
                  os.fsync(f.fileno())
                  prog.value += len(data)
            else :
              print('Somente um arquivo pode ser enviado!')
      except Exception as e:
          teste = e
  # Determinando o arquivo que será importado

  uploaded_file = widgets.FileUpload(
      accept='.pdf',
      disabled=False,
      multiple=False)

  button = widgets.Button(
      description='Enviar',
      button_style='success')

  threading.Thread(target=uploaded_file.observe(on_button_clicked)).start()

  display(uploaded_file)

In [ ]:
def download_arquivo(nome_arquivo):
  from google.colab import files

  if (nome_arquivo == ''):
    print('Precisava digitar o nome do arquivo!')
  else:
      # Caminho completo do arquivo na pasta content
      caminho_do_arquivo = f"/content/{nome_arquivo}"
      # Baixar o arquivo
      files.download(caminho_do_arquivo)


In [ ]:
nome_arquivo  = input('Qual o nome do arquivo: ')
threading.Thread(target=download_arquivo,args=(nome_arquivo,)).start()

In [ ]:
upload = {
    "funcao": "upload_arquivo",
    "parametros": "uploaded_file, upload, button"}

download = {
    "funcao": "download_arquivo",
    "parametros": "download_file, download, file, filename"}
chat = {
    "funcao": "iniciar_chat",
    "parametros": "gemini, chat, pesquisa, sobre, assunto, busca"
}
documents = [upload, download, chat]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["funcao", "parametros"]
df

In [ ]:
modelo_funcao = "models/embedding-001"
def embed_fn(title, text):
  return genai.embed_content(model=modelo_funcao,
                             content=text,
                             title=title,
                             task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["funcao"], row["parametros"]), axis=1)
df

In [ ]:
def buscar_a_funcao_exibir(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)
  indice = np.argmax(produtos_escalares)
  print(df.iloc[indice]["funcao"])
  if (df.iloc[indice]["funcao"] == 'upload_arquivo'):
    print('Pode agora subir com os seus arquivos!')
    upload_arquivo()
    return 1
  else:
    if (df.iloc[indice]["funcao"] == 'download_arquivo'):
        nome_arquivo  = input('Qual o nome do arquivo: ')
        threading.Thread(target=download_arquivo,args=(nome_arquivo,)).start()
        return 1
    else:
         #print('Vou buscar no gemini!')
         return 0


In [ ]:
consulta = "quero fazer um upload?"

buscar_a_funcao_exibir(consulta, df, modelo_funcao)
#print(retorno)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
model = genai.GenerativeModel(model_name='gemini-1.5-pro-latest',
                              safety_settings=safety_settings,
                              generation_config=generation_config,)

In [ ]:
chat = model.start_chat(history=[])

In [ ]:
def iniciar_chat():
  try:
    texto = input('Digite a sua pergunta: ')

    while texto != "fim":
      info = texto
      if (buscar_a_funcao_exibir(texto, df, modelo_funcao)) == 1:
        texto = 'fim'
      else:
        response = chat.send_message(texto)
        print(response.text)
      texto = input('Digite a sua pergunta: ')
  except Exception as e:
      print(e)

In [ ]:
iniciar_chat()